In [1]:
# Import library for load data
import ray
ray.init(num_cpus=8)
import modin.pandas as pd
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv
# Import deeplearning library
import time
import os
import glob
import copy
import numpy as np 
import argparse
import sys
sys.path.append('/home/jun/DDoSDeepLearningProject/')
from CustomFunction import *
from model_define import *


2023-07-28 18:48:01,018	INFO worker.py:1636 -- Started a local Ray instance.


In [2]:
#----------------PARSE PARAMETER-----------------
parser = argparse.ArgumentParser(description='This program will take raw dataset of CICIDS2017, CICIDS2018, CICIDS2019 and turn it into clean all_attack dataset and ddos only dataset')
# parser.add_argument: Add parameter to program
WORK_PATH = os.getcwd()
parser.add_argument('--dataset', help='Raw dataset folder', default= '/home/jun/DDoSDeepLearningProject/Dataset')
parser.add_argument('--fixed', help='Fixed dataset folder', default='/home/jun/DDoSDeepLearningProject/dataset_all_attack') 
parser.add_argument('--ddosonly', help='DDoS only dataset folder', default='/home/jun/DDoSDeepLearningProject/dataset_ddos') 
#args = parser.parse_args()
args = parser.parse_args(args=['--dataset','/home/jun/DDoSDeepLearningProject/Dataset', '--fixed', '/home/jun/DDoSDeepLearningProject/dataset_all_attack', '--ddosonly', '/home/jun/DDoSDeepLearningProject/dataset_ddos'])
#----------------Define variable based of args----------

raw_dataset_PATH = args.dataset
fixed_dataset_PATH = args.fixed
ddos_dataset_PATH =  args.ddosonly
os.makedirs(fixed_dataset_PATH, exist_ok=True)
os.makedirs(ddos_dataset_PATH, exist_ok=True)
# Make folder for validate data file
print('-------------PARAMETERS-------------')
print('Raw dataset folder PATH:',raw_dataset_PATH)
print('Fixed dataset (all attack) PATH:', fixed_dataset_PATH)
print('DDoS dataset (DDoS attack only) PATH:', ddos_dataset_PATH)

-------------PARAMETERS-------------
Raw dataset folder PATH: /home/jun/DDoSDeepLearningProject/Dataset
Fixed dataset (all attack) PATH: /home/jun/DDoSDeepLearningProject/dataset_all_attack
DDoS dataset (DDoS attack only) PATH: /home/jun/DDoSDeepLearningProject/dataset_ddos


In [3]:
folder = raw_dataset_PATH
csv_file = csv_file_list(folder)
for folder in csv_file.keys():
    print(folder)
    for file in csv_file[folder]:
        print(f'\t{file}')

CICIDS2019
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_UDP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/Syn.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_NetBIOS.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/UDPLag.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/TFTP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_LDAP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_MSSQL.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_SSDP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_NTP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_SNMP.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_DNS.csv
CICIDS2017
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2017/Monday-WorkingHours.pcap_ISCX.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2017/Wednesday-workingHours.pcap_ISCX.csv
	/home/jun/DDoSDeepLearningProject/Dataset/CICIDS2017/Fr

In [4]:
CICIDS2019_columns = ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'SimillarHTTP', 'Inbound', 'Label']

CICIDS2018_columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port',
                'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
                'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
                'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
                'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
                'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
                'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
                'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
                'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
                'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
                'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
                'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
                'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
                'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
                'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
                'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
                'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
                'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
                'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
                'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
                'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2018_columns_fixed = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2018_columns_fixed_2 = [
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2017_columns = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

print("CICIDS2019 columns:", len(CICIDS2019_columns))
print("CICIDS2018 columns: {} and {}".format(len(CICIDS2018_columns_fixed), len(CICIDS2018_columns_fixed_2)))
print("CICIDS2017 columns:", len(CICIDS2017_columns))

CICIDS2019 columns: 88
CICIDS2018 columns: 84 and 80
CICIDS2017 columns: 85


In [5]:
print('-------------ANALYZE DATASET-------------')
data = {}
for dataset in csv_file.keys():
    data_dict = {}
    print(f"Start loading folder {dataset}")
    for file in csv_file[dataset]:
        print(f"\tLoading {file}")
        data_key =file.split('/')[-1].split('.')[0] 
        print(data_key)
        data_dict.update({file.split('/')[-1].split('.')[0]:pd.read_csv(file, encoding='cp1252')})
        print("\tDONE!")
    data.update({dataset:data_dict})
print("ALL DONE!")
    

-------------ANALYZE DATASET-------------
Start loading folder CICIDS2019
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_UDP.csv
DrDoS_UDP


	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/Syn.csv
Syn
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_NetBIOS.csv
DrDoS_NetBIOS
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/UDPLag.csv
UDPLag


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/TFTP.csv
TFTP
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_LDAP.csv
DrDoS_LDAP
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_MSSQL.csv
DrDoS_MSSQL
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_SSDP.csv
DrDoS_SSDP
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_NTP.csv
DrDoS_NTP
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_SNMP.csv
DrDoS_SNMP
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2019/DrDoS_DNS.csv
DrDoS_DNS
	DONE!
Start loading folder CICIDS2017
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2017/Monday-WorkingHours.pcap_ISCX.csv
Monday-WorkingHours
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2017/Wednesday-workingHours.pcap_ISCX.csv
Wednesday-workingHours
	DONE!
	Loading /home/jun/DDoSDeepLearningProject/Dataset/CICIDS2

In [6]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print(f"\t {csv_name}.csv have {len(csv_data.columns)} columns")

CICIDS2019
	 DrDoS_UDP.csv have 88 columns
	 Syn.csv have 88 columns
	 DrDoS_NetBIOS.csv have 88 columns
	 UDPLag.csv have 88 columns
	 TFTP.csv have 88 columns
	 DrDoS_LDAP.csv have 88 columns
	 DrDoS_MSSQL.csv have 88 columns
	 DrDoS_SSDP.csv have 88 columns
	 DrDoS_NTP.csv have 88 columns
	 DrDoS_SNMP.csv have 88 columns
	 DrDoS_DNS.csv have 88 columns
CICIDS2017
	 Monday-WorkingHours.csv have 85 columns
	 Wednesday-workingHours.csv have 85 columns
	 Friday-WorkingHours-Morning.csv have 85 columns
	 Friday-WorkingHours-Afternoon-PortScan.csv have 85 columns
	 Friday-WorkingHours-Afternoon-DDos.csv have 85 columns
	 Thursday-WorkingHours-Morning-WebAttacks.csv have 85 columns
	 Tuesday-WorkingHours.csv have 85 columns
	 Thursday-WorkingHours-Afternoon-Infilteration.csv have 85 columns
CICIDS2018
	 Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv have 84 columns
	 Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv have 80 columns
	 Friday-02-03-2018_TrafficForML_CICFlowMeter.csv have

In [7]:
# Fix columns name of all dataset
for dataset, data_dict in data.items():
    for csv_name, csv_data in data_dict.items():
        csv_data = fix_columns_name(csv_data)
for csv_name, csv_data in data['CICIDS2018'].items():
    try:
        csv_data.columns = CICIDS2018_columns_fixed
    except:
        csv_data.columns = CICIDS2018_columns_fixed_2

In [8]:
print('Drop Thursday-WorkingHours-Morning-WebAttacks because of error label')
del data['CICIDS2017']['Thursday-WorkingHours-Morning-WebAttacks']
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print(f"\t {csv_name}.csv have labels: {set(csv_data['Label'])}")
        

Drop Thursday-WorkingHours-Morning-WebAttacks because of error label
CICIDS2019
	 DrDoS_UDP.csv have labels: {'BENIGN', 'DrDoS_UDP'}
	 Syn.csv have labels: {'Syn', 'BENIGN'}
	 DrDoS_NetBIOS.csv have labels: {'DrDoS_NetBIOS', 'BENIGN'}
	 UDPLag.csv have labels: {'BENIGN', 'UDP-lag', 'WebDDoS'}
	 TFTP.csv have labels: {'BENIGN', 'TFTP'}
	 DrDoS_LDAP.csv have labels: {'BENIGN', 'DrDoS_LDAP'}
	 DrDoS_MSSQL.csv have labels: {'BENIGN', 'DrDoS_MSSQL'}
	 DrDoS_SSDP.csv have labels: {'DrDoS_SSDP', 'BENIGN'}
	 DrDoS_NTP.csv have labels: {'DrDoS_NTP', 'BENIGN'}
	 DrDoS_SNMP.csv have labels: {'BENIGN', 'DrDoS_SNMP'}
	 DrDoS_DNS.csv have labels: {'BENIGN', 'DrDoS_DNS'}
CICIDS2017
	 Monday-WorkingHours.csv have labels: {'BENIGN'}
	 Wednesday-workingHours.csv have labels: {'DoS Slowhttptest', 'DoS slowloris', 'Heartbleed', 'BENIGN', 'DoS Hulk', 'DoS GoldenEye'}
	 Friday-WorkingHours-Morning.csv have labels: {'Bot', 'BENIGN'}
	 Friday-WorkingHours-Afternoon-PortScan.csv have labels: {'PortScan', 'BENI

In [9]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        this_columns = csv_data.columns
        not_included = []
        for column in CICIDS2019_columns:
            if column not in this_columns:
                not_included.append(column)
        print(f"\t{csv_name},csv not included {len(not_included)} columns: {not_included}")

CICIDS2019
	DrDoS_UDP,csv not included 0 columns: []
	Syn,csv not included 0 columns: []
	DrDoS_NetBIOS,csv not included 0 columns: []
	UDPLag,csv not included 0 columns: []
	TFTP,csv not included 0 columns: []
	DrDoS_LDAP,csv not included 0 columns: []
	DrDoS_MSSQL,csv not included 0 columns: []
	DrDoS_SSDP,csv not included 0 columns: []
	DrDoS_NTP,csv not included 0 columns: []
	DrDoS_SNMP,csv not included 0 columns: []
	DrDoS_DNS,csv not included 0 columns: []
CICIDS2017
	Monday-WorkingHours,csv not included 3 columns: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	Wednesday-workingHours,csv not included 3 columns: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	Friday-WorkingHours-Morning,csv not included 3 columns: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	Friday-WorkingHours-Afternoon-PortScan,csv not included 3 columns: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	Friday-WorkingHours-Afternoon-DDos,csv not included 3 columns: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	Tuesday-WorkingHours

In [10]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        this_columns = csv_data.columns
        not_included = []
        for column in this_columns:
            if column not in CICIDS2018_columns_fixed_2:
                not_included.append(column)
        print(f"\t Drop {len(not_included)} columns: {not_included} from {csv_name}.csv")

CICIDS2019
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from DrDoS_UDP.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from Syn.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from DrDoS_NetBIOS.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from UDPLag.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from TFTP.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound'] from DrDoS_LDAP.csv
	 Drop 8 columns: ['Unnamed: 0', 'Flow ID', 

In [11]:
CICIDS2019_drop_list = ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
CICIDS2018_drop_list = ['Destination Port', 'Timestamp']
CICIDS2018_drop_list_2 = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp']
CICIDS2017_drop_list = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Fwd Header Length.1']

In [36]:
for csv_name, csv_data in data['CICIDS2018'].items():
    try:
        csv_data.drop(CICIDS2018_drop_list_2, inplace=True, axis=1)
        print(f'{csv_name} have: {len(csv_data.columns)} columns')
    except:
        csv_data.drop(CICIDS2018_drop_list, inplace=True, axis=1)
        print(f"Error {csv_name}")


Thuesday-20-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Wednesday-14-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Friday-02-03-2018_TrafficForML_CICFlowMeter have: 78 columns
Wednesday-21-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Friday-23-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Wednesday-28-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Thursday-15-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Friday-16-02-2018_TrafficForML_CICFlowMeter have: 78 columns
Thursday-01-03-2018_TrafficForML_CICFlowMeter have: 78 columns
Thursday-22-02-2018_TrafficForML_CICFlowMeter have: 78 columns


In [21]:
for csv_name, csv_data in data['CICIDS2017'].items():
    try:
        csv_data.drop(CICIDS2017_drop_list, inplace=True, axis=1)
        print(f'{csv_name} have: {len(csv_data.columns)} columns')
    except:
        print(f"Error {csv_name}")

Monday-WorkingHours have: 78 columns
Wednesday-workingHours have: 78 columns
Friday-WorkingHours-Morning have: 78 columns
Friday-WorkingHours-Afternoon-PortScan have: 78 columns
Friday-WorkingHours-Afternoon-DDos have: 78 columns
Tuesday-WorkingHours have: 78 columns
Thursday-WorkingHours-Afternoon-Infilteration have: 78 columns


In [ ]:
for csv_name, csv_data in data['CICIDS2019'].items():
    try:
        csv_data.drop(CICIDS2019_drop_list, inplace=True, axis=1)
    except:
        print(f"Error {csv_name}")

In [ ]:
merged = {}
for dataset, data_dict in data.items():
    merged.update({dataset:pd.concat(data_dict.values(), ignore_index=True, sort=False)})

Writing to CICIDS2019
Writing to CICIDS2017
Writing to CICIDS2018


In [ ]:
CICIDS2018_labels = {'DoS attacks-SlowHTTPTest':'DoS Slowhttptest', 'DoS attacks-GoldenEye':'DoS GoldenEye', 'DoS attacks-Slowloris':'DoS slowloris','DoS attacks-Hulk':'DoS Hulk', 'Benign': 'BENIGN'}
for label_old, label_new in CICIDS2018_labels.items():
        merged['CICIDS2018']['Label'] = merged['CICIDS2018']['Label'].replace(label_old, label_new)

In [ ]:
for dataset, data_merged in merged.items():
    if dataset == 'CICIDS2018':
        merged['CICIDS2018'] = merged['CICIDS2018'].loc[merged['CICIDS2018']['Label'].isin('Label')]
    print(f"\t {dataset} (all_type_attack) have labels: {set(data_merged['Label'])}")

	 CICIDS2019 (all_type_attack) have labels: {'DrDoS_MSSQL', 'WebDDoS', 'Syn', 'DrDoS_SNMP', 'DrDoS_LDAP', 'DrDoS_DNS', 'BENIGN', 'DrDoS_NetBIOS', 'UDP-lag', 'DrDoS_NTP', 'DrDoS_SSDP', 'TFTP', 'DrDoS_UDP'}
	 CICIDS2017 (all_type_attack) have labels: {'DDoS', 'Heartbleed', 'Infiltration', 'DoS GoldenEye', 'Bot', 'FTP-Patator', 'BENIGN', 'SSH-Patator', 'DoS slowloris', 'DoS Hulk', 'DoS Slowhttptest', 'PortScan'}
	 CICIDS2018 (all_type_attack) have labels: {'DDOS attack-HOIC', 'Bot', 'DoS GoldenEye', 'Brute Force -XSS', 'Label', 'Infilteration', 'Brute Force -Web', 'BENIGN', 'DoS Slowhttptest', 'DDoS attacks-LOIC-HTTP', 'DoS slowloris', 'FTP-BruteForce', 'SSH-Bruteforce', 'DoS Hulk', 'DDOS attack-LOIC-UDP', 'SQL Injection'}


In [ ]:
ddos_labels = {
    'CICIDS2019': ['Syn', 'DrDoS_NetBIOS', 'DrDoS_SNMP', 'DrDoS_DNS', 'DrDoS_LDAP', 'DrDoS_UDP', 'UDP-lag', 'DrDoS_NTP', 'BENIGN', 'WebDDoS', 'DrDoS_MSSQL', 'DrDoS_SSDP', 'TFTP'],
    'CICIDS2018' : ['DDOS attack-LOIC-UDP', 'BENIGN', 'DoS Slowhttptest', 'DoS slowloris', 'DoS GoldenEye', 'DDOS attack-HOIC', 'DoS Hulk', 'DDoS attacks-LOIC-HTTP'],
    'CICIDS2017' : ['BENIGN', 'DoS Slowhttptest', 'DoS slowloris', 'DoS GoldenEye', 'DDoS', 'DoS Hulk']
}
ddos_only = {}
for dataset, data_ddos in merged.items():
    ddos_only.update({dataset:data_ddos.loc[data_ddos['Label'].isin(ddos_labels[dataset])]})

2023-07-28 16:37:33,521	WARNING worker.py:2019 -- WARNING: 32 PYTHON worker processes have been started on node: f139b3d6ed8a910cf5b64812d90245752783037a5f207416f9750c3f with address: 202.25.82.68. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).


In [ ]:
for dataset, data_merged in ddos_only.items():
    print(f"\t {dataset} (ddos_only) have labels: {set(data_merged['Label'])}")

	 CICIDS2019 (ddos_only) have labels: {'DrDoS_MSSQL', 'WebDDoS', 'Syn', 'DrDoS_SNMP', 'DrDoS_LDAP', 'DrDoS_DNS', 'BENIGN', 'DrDoS_NetBIOS', 'UDP-lag', 'DrDoS_NTP', 'DrDoS_SSDP', 'TFTP', 'DrDoS_UDP'}
	 CICIDS2017 (ddos_only) have labels: {'DDoS', 'DoS GoldenEye', 'BENIGN', 'DoS slowloris', 'DoS Hulk', 'DoS Slowhttptest'}
	 CICIDS2018 (ddos_only) have labels: {'DDOS attack-HOIC', 'DoS GoldenEye', 'BENIGN', 'DoS Slowhttptest', 'DDoS attacks-LOIC-HTTP', 'DoS slowloris', 'DoS Hulk', 'DDOS attack-LOIC-UDP'}


In [ ]:
for dataset, data_merged in merged.items():
    print('Writing to {}'.format(dataset))
    merged_PATH =fixed_dataset_PATH + '/' + dataset + '.csv'
    print(f'Writing to {merged_PATH}')
    data_merged.to_csv(merged_PATH,index=False) 
    print('ALL DONE!')

Writing to CICIDS2019
Writing to /home/jun/DDoSDeepLearningProject/dataset_all_attackCICIDS2019.csv


(raylet) Spilled 61912 MiB, 813 objects, write throughput 899 MiB/s.
(raylet) Spilled 63396 MiB, 833 objects, write throughput 907 MiB/s.
(raylet) Spilled 67784 MiB, 894 objects, write throughput 930 MiB/s.


ALL DONE!
Writing to CICIDS2017
Writing to /home/jun/DDoSDeepLearningProject/dataset_all_attackCICIDS2017.csv
ALL DONE!
Writing to CICIDS2018
Writing to /home/jun/DDoSDeepLearningProject/dataset_all_attackCICIDS2018.csv


(raylet) Spilled 138464 MiB, 1339 objects, write throughput 1026 MiB/s.


ALL DONE!


In [ ]:
for dataset, data_merged in ddos_only.items():
    print('Writing to {}'.format(dataset))
    merged_PATH =ddos_dataset_PATH + '/' + dataset + '.csv'
    print(f'Writing to {merged_PATH}')
    data_merged.to_csv(merged_PATH,index=False) 
    print('ALL DONE!')